**Task 1** 
***
* Download the monthly natural gas price data. Each point in the data set corresponds to the purchase price of natural gas at the end of a month, from 31st October 2020 to 30th September 2024.
* Analyze the data to estimate the purchase price of gas at any date in the past and extrapolate it for one year into the future.

   
Objective: The code should take a date as input and return a price estimate.

***
    
Given the characteristics of the data (non-stationary, positive linear trend, yearly seasonality), I have two options:
* Triple Exponential Smoothing (also known as Holt-Winter's model) - used to forecast data with seasonality and trend. Since I have monthly granularity, I need to forecast 12 data points, and this means this model will be appropriate since it is used for short to medium-term forecasts.
* Sarima Model - is used to forecast data with seasonality and trend, and also because the statistical properties like mean and variance change over time.


I will apply both to see the forecasting accuracy using RMSE (Root Mean Square Error).
The lower this value, the better the model. Triple Exponential Smoothing performs better


**Task 2**
***

Write a script that can be used to price the contract.
You need to create a prototype pricing model.

You should write a function that can use the data you created previously (output of the forecasting model) to price the contract. The client may want to choose multiple dates to inject and withdraw a set amount of gas, so your approach should generalize the explanation from before. Consider all the cash flows involved in the product.

The input parameters that should be taken into account for pricing are:

* Injection dates. 
* Withdrawal dates.
* The prices at which the commodity can be purchased/sold on those dates.
* The rate at which the gas can be injected/withdrawn.
* The maximum volume that can be stored.
* Storage costs.
  
Write a function that takes these inputs and gives back the value of the contract. You can assume there is no transport delay and that interest rates are zero. Market holidays, weekends, and bank holidays need not be accounted for. Test your code by selecting a few sample inputs.

The pricing model is at the end of the notebook, please run all cells before using it.


### Import data and libraries

In [ ]:
from datetime import datetime
from math import sqrt

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import scipy.stats as stats
import statsmodels.api as sm
from pmdarima.arima import auto_arima
from pmdarima.arima.utils import ndiffs
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

In [ ]:
# Import data
dataset = pd.read_csv("Data/Nat_Gas.csv")
dataset.head()

In [ ]:
# Check data types
dataset.info()

In [ ]:
# Convert the 'Dates' column to a datetime type
dataset["Dates"] = pd.to_datetime(dataset["Dates"], format="%m/%d/%y")

In [ ]:
# Double check result
dataset.info()

In [ ]:
dataset.head()

In [ ]:
# Set 'Dates' column as index
dataset.set_index("Dates", inplace=True)

In [ ]:
# Since I have time series data I want to visualise them so see if there are any pattern, outliears and also
# before doing a seasonall decomposing I want to understand whether the seasonality shows multiplicative or additive behavior

In [ ]:
dataset.plot()

In [ ]:
# From the plot I can observe a positive linear trend with an addittive seasonality

In [ ]:
# Ho have an idea of the data I want to see mean and variance of data to understand if the data is stationary.
# Later I will run the Augmented Dickey-Fuller test to verify if the data are stationary

In [ ]:
dataset.rolling(window=6).mean().plot()
# Already from this plot I can see that the data is not stationary since the mean is varying over different time periods

In [ ]:
# I want to know more about the data so I'm going to display basic statistics.
dataset.describe()

In [ ]:
dataset.skew()

In [ ]:
dataset.kurtosis()

In [ ]:
decomposed_gas_price = seasonal_decompose(dataset, period=13, model="additive")

fig, ax = plt.subplots(4, 1, figsize=(16, 10))
ax[0].plot(decomposed_gas_price.observed)
ax[0].set_ylabel("Observed")
ax[0].set_title("Decomposition for Multiplicative Model")

ax[1].plot(decomposed_gas_price.trend)
ax[1].set_ylabel("Trend")

ax[2].plot(decomposed_gas_price.seasonal, label="Seasonal")
ax[2].set_ylabel("Seasonal")

ax[3].plot(decomposed_gas_price.resid, label="Residual")
ax[3].set_ylabel("Residual")

plt.legend()
plt.show()

In [ ]:
# Augmented Dickey-Fuller  unit root test
def ADF_test(data):
    """
    Perform Augmented Dickey-Fuller Test
    The number of considered lags is automatically selected based
    on the Akaike Information Criterion (AIC)
    Ho = Null Hypothesis -> The data are not stationary
    H1 = Alternate Hypotesis -> The data are stationary
    """
    result = adfuller(data, autolag="AIC")
    print(f"ADF Statistic:{result[0]}")
    print(f"P-value: {result[1]}")
    print("Critical Values:")
    for x, y in result[4].items():
        print(f"{x}:{y}")
    if result[1] > 0.05:
        print("We accept the Null Hypotesis. Data are not stationary!")
    else:
        print("We accept the Alternate Hypotesis. Data are Stationary!")

In [ ]:
ADF_test(dataset)

In [ ]:
# Find the optimal number of differencing
number_of_diff = ndiffs(dataset, test="adf")
print(f"Number of differencing: {number_of_diff}")

In [ ]:
# I need to take the 12th difference for monthly data
first_order_diff = dataset.diff(12).dropna()

In [ ]:
first_order_diff.plot()

In [ ]:
ADF_test(first_order_diff)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
plot_acf(first_order_diff, zero=False, ax=ax[0])
plot_pacf(first_order_diff, zero=False, ax=ax[1])
plt.show()

In [ ]:
# first_order_diff
split = int(len(dataset) * 0.93)
train_data = dataset.iloc[:split]
test_data = dataset.iloc[split:]

In [ ]:
# Start_p -> the order of the auto-regressive (“AR”) model
# Start_q-> the starting value of q, the order of the moving-average (“MA”) model.
# m -> the number of periods in each season
model = auto_arima(
    train_data,
    start_p=1,
    start_q=1,
    max_p=12,
    max_q=12,
    m=12,
    start_P=0,
    seasonal=True,
    # d=1, D=1,
    trace=True,
    error_action="ignore",
    test="adf",
    information_criterion="aic",
    suppress_warnings=True,
)

In [ ]:
model.summary()

In [ ]:
# Residuals to check their normality
output = model.plot_diagnostics(figsize=(16, 7))
# correlogram tells us that there is no any pattern left in the residuals, so this is just noise

In [ ]:
test_data

In [ ]:
auto_arima_fitted = model.fit(train_data)

In [ ]:
auto_arima_prediction = auto_arima_fitted.predict(len(test_data) + 12).to_frame(
    name="Prices"
)

In [ ]:
auto_arima_prediction

In [ ]:
auto_arima_rmse = sqrt(
    mean_squared_error(test_data, auto_arima_prediction.iloc[: len(test_data), :])
)
print("RMSE: ", auto_arima_rmse)

In [ ]:
plt.figure(figsize=(16, 7))
plt.plot(train_data, c="royalblue")
plt.plot(test_data, c="green", ls="-")
plt.plot(auto_arima_prediction, c="grey", ls="--")
plt.xlabel("Date", size=20)
plt.ylabel("Market Price", size=20)
plt.title("Market Price Prediction by Auto ARIMA", size=20)
plt.legend(["Train Data", "Observed Data", "Predicted Data"])
plt.grid(linestyle="--", c="grey")

* Lets try triple exponencial smoothing

In [ ]:
dataset.tail()

In [ ]:
# Devide the dataset to train (data used to train the model) and test dataset (used to validate the model)
split = int(len(dataset) * 0.80)
train_data_es = dataset.iloc[:split]
test_data_es = dataset.iloc[split:]

In [ ]:
seasonal_periods = 12
# Holt-Winter's model with exponential trend
hw_1 = ExponentialSmoothing(
    train_data_es,
    trend="add",
    seasonal="add",
    seasonal_periods=seasonal_periods,
    damped_trend=False,
    freq="M",
).fit()

In [ ]:
# visualiss model's parameters
hw_1.params

In [ ]:
hw_forecast_1 = hw_1.forecast(len(test_data_es) + 12)

In [ ]:
hw_forecast_1 = hw_forecast_1.to_frame(name="Prices")

In [ ]:
plt.figure(figsize=(16, 7))
plt.plot(train_data_es, c="royalblue")
plt.plot(test_data_es, c="green", ls="-")
plt.plot(hw_forecast_1, c="grey", ls="--")
plt.xlabel("Date", size=20)
plt.ylabel("Market Price", size=20)
plt.title("Market Price Holt-Winter's Seasonal Smoothing", size=20)
plt.legend(["Train Data", "Observed Data", "Predicted Data"])
plt.grid(linestyle="--", c="grey")

In [ ]:
rmse_triple_exp_smoothing = np.sqrt(
    mean_squared_error(test_data_es, hw_forecast_1.iloc[:-12])
)

print(f"Triple Exponential Smoothing RMSE: {rmse_triple_exp_smoothing}")

In [ ]:
df = pd.concat([dataset, hw_forecast_1.iloc[len(test_data) :, :]])

In [ ]:
hw_1.params

In [ ]:
alpha = hw_1.params["smoothing_level"]
beta = hw_1.params["smoothing_trend"]
gamma = hw_1.params["smoothing_seasonal"]

## Enter date to get forecasted price

In [ ]:
def forecast_price():
    user_date_input = input(
        "Forecast starts from '2024-10-31'.Enter a end month date in 'YYYY-MM-DD' format: "
    )
    return df.loc[user_date_input, :]

In [ ]:
forecast_price()

# Pricing Model

In [ ]:
def pricing_model(
    *,
    units=1000000,
    purchase_date="2023-06-30",
    sell_date="2024-01-31",
    storage_cost_per_month_1m_units=20000,
    max_capacity_units=3000000,
    inj_withdr_cost_per_1mil_units=10000,
    transport_cost_one_way=1000,
    pricing_dataframe
):
    """
    Input: DataFrame with dates and prices for unit of commodity

    Parameters:
    units: number of good units purchased
    purchase_date: purchase date
    sell_date: sell date
    storage_cost_per_month_1m_units: storage cost for 1 million units
    max_capacity_units: max storage capacity
    inj_withdr_cost_per_1mil_units: sum of injection and withdrawal cost per 1 million units
    transport_cost_one_way: transport cost one way
    Output: Value of a contract
    """

    if units <= max_capacity_units:
        number_of_months = int(
            (
                datetime.strptime(sell_date, "%Y-%m-%d").year
                - datetime.strptime(purchase_date, "%Y-%m-%d").year
            )
            * 12
            + (
                datetime.strptime(sell_date, "%Y-%m-%d").month
                - datetime.strptime(purchase_date, "%Y-%m-%d").month
            )
        )
        # Dataset with Pricing
        df = pricing_dataframe
        # Total revenue
        total_revenue = units * df.loc[sell_date, "Prices"]
        # Total costs based on months

        # Initial cost of purchase -> fixed costs
        fixed_costs = (units * df.loc[purchase_date, "Prices"]) + (
            transport_cost_one_way * 2
        )
        # Variable costs based on the number of units purchased
        total_cost_for_storage = (
            (storage_cost_per_month_1m_units / 1000000) * units * number_of_months
        )
        inj_withdr_cost = (inj_withdr_cost_per_1mil_units / 1000000) * units

        variable_cost = total_cost_for_storage + inj_withdr_cost
        total_cost = fixed_costs + variable_cost

        revenue_loss = {"Revenue": 0.00, "Loss": 0.00}

        if total_revenue > total_cost:
            profit = total_revenue - total_cost
            revenue_loss["Revenue"] = profit
        elif total_revenue < total_cost:
            loss = total_revenue - total_cost
            revenue_loss["Loss"] = loss
        elif total_revenue == total_cost:
            neutral = total_revenue - total_cost
        result = pd.DataFrame(revenue_loss, index=[1]).T.rename(columns={1: "£ Value"})

    else:
        result = "Order is out of capacity!"
    return result

In [ ]:
pricing_model(pricing_dataframe=df)